In [1]:
from Core.filter import RdrFilter
explorer = RdrFilter()

RDRCUMINDEX.TAB already exists. Set `reload = True` to force download.
RDRCUMINDEX.LBL already exists. Set `reload = True` to force download.


In [2]:
explorer.load_df()

NO FILTER ............................................. 93425 images


In [3]:
# explorer.latitude_filter(commit=False)
# explorer.visualize(engine = 'pygmt', target = 'img_rectangle', filename = 'latitude_flt')

explorer.latitude_filter()

LATITUDE FILTER ........................................ 5904 images


In [4]:
# explorer.scale_filter(commit = False)
# explorer.visualize(engine = 'pygmt', target = 'img_rectangle', filename = 'scale_flt')

explorer.scale_filter()

SCALE FILTER ........................................... 3699 images


In [6]:
# explorer.season_filter(commit = False)
# explorer.visualize(engine = 'pygmt', target = 'img_rectangle', filename = 'season_flt')
# explorer.visualize(engine = 'qgis', target = 'img_rectangle', filename = 'season_flt')

explorer.season_filter()

SEASON FILTER .......................................... 2081 images


In [7]:
explorer.cluster_filter(commit = False)
# explorer.visualize(engine = 'pygmt', target = 'cluster_centroid', filename = 'cluster_flt')
explorer.visualize(engine = 'qgis', target = 'img_rectangle', filename = 'cluster_flt')

CLUSTER FILTER ......................................... 1503 images


In [ ]:
explorer.keyword_filter(commit = False)
explorer.visualize(engine = 'pygmt', target = 'cluster_centroid', filename = 'keyword_flt')

In [ ]:
explorer.my_filter(commit = False)
explorer.visualize(engine = 'pygmt', target = 'cluster_centroid', filename = 'my_flt')

In [ ]:
explorer.allignment_filter(commit = False)
explorer.visualize(engine = 'pygmt', target = 'cluster_centroid', filename = 'allignment_flt')

ALLIGNMENT FILTER ....................................... 203 images


In [8]:
print(explorer.local_conf)

{'latitude_filter': {'min_lat': 78}}


In [11]:
explorer.__dict__.keys()

dict_keys(['local_conf', 'df', 'tmp_df', 'mapper'])